### Data sampling

In [1]:
from functions import SampleImages

SampleImages(num_train=1000, num_valid=1000, num_test=100)

 82%|████████▏ | 9/11 [00:00<00:00, 79.05it/s]

Clearing the folders...


  2%|▏         | 23/1000 [00:00<00:04, 220.86it/s]

Copying the images...


100%|██████████| 1000/1000 [00:04<00:00, 241.37it/s]


In [2]:
from keras import models
from keras import layers
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

### Load train and test dataset

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_generator = ImageDataGenerator(validation_split=0.2)

training_generator = data_generator.flow_from_directory(
    'train-small', 
    classes=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged',
          'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'], 
    batch_size=100,
    subset='training',
    color_mode='grayscale'
)
validation_generator = data_generator.flow_from_directory(
    'validation-small', 
    classes=['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged',
          'NGT - Normal', 'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'], 
    batch_size=100,
    subset='validation',
    color_mode='grayscale'
)


Found 1325 images belonging to 11 classes.
Found 343 images belonging to 11 classes.


In [7]:
images, labels = training_generator.next()
print(images.shape)
print(labels.shape)

(100, 256, 256, 1)
(100, 11)


In [8]:
images, labels = validation_generator.next()
print(images.shape)
print(labels.shape)

(100, 256, 256, 1)
(100, 11)


### define the structure of network

In [9]:
import tensorflow.keras as keras

'''
# this model takes too long to train
model = keras.Sequential([
    # First Convolutional Block
    layers.Conv2D(filters=32, kernel_size=5, activation="relu", padding='same',
                  # give the input dimensions in the first layer
                  # [height, width, color channels(RGB)]
                  input_shape=[256, 256, 1]),
    layers.MaxPool2D(),

    # Second Convolutional Block
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Third Convolutional Block
    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(),

    # Classifier Head
    layers.Flatten(),
    layers.Dense(units=4, activation="relu"),
    layers.Dense(units=4, activation="sigmoid"),
])
'''
model = keras.Sequential([
    layers.Conv2D(16, (5, 5), activation='relu', padding='same', input_shape=[256, 256, 1]),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(11, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
flatten (Flatten)            (None, 262144)            0         
_________________________________________________________________
dense (Dense)                (None, 11)                2883595   
Total params: 2,884,011
Trainable params: 2,884,011
Non-trainable params: 0
_________________________________________________________________


### train the model

In [10]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy', 
    metrics=['accuracy'],
)

history = model.fit(
    training_generator,
    validation_data = validation_generator,
    epochs=5,
)


Epoch 1/5
14/14 [==============================] - 32s 2s/step - loss: 2711.3311 - accuracy: 0.1917 - val_loss: 977.4109 - val_accuracy: 0.4052
Epoch 2/5
14/14 [==============================] - 28s 2s/step - loss: 280.3224 - accuracy: 0.3343 - val_loss: 31.2526 - val_accuracy: 0.3732
Epoch 3/5
14/14 [==============================] - 30s 2s/step - loss: 14.0492 - accuracy: 0.4075 - val_loss: 15.0369 - val_accuracy: 0.2886
Epoch 4/5
14/14 [==============================] - 27s 2s/step - loss: 7.5604 - accuracy: 0.4166 - val_loss: 12.7880 - val_accuracy: 0.3703
Epoch 5/5
14/14 [==============================] - 28s 2s/step - loss: 4.3828 - accuracy: 0.4830 - val_loss: 10.0283 - val_accuracy: 0.3382


### save the model

In [16]:

model.save("models/basic_small.nn")


INFO:tensorflow:Assets written to: models/basic_small.nn/assets


In [17]:
model = keras.models.load_model("models/basic_small.nn")

### Evaluation

In [18]:
val_loss, val_acc = model.evaluate(validation_generator)

print(f"Evaluation result on Test Data : Loss = {val_loss}, accuracy = {val_acc}")

4/4 [==============================] - 4s 885ms/step - loss: 10.0283 - accuracy: 0.3382
Evaluation result on Test Data : Loss = 10.028291702270508, accuracy = 0.3381924331188202


### Prediction

In [21]:
pred = model.predict(validation_generator)

In [23]:
pred.shape

(343, 11)

In [24]:
pred[0]

array([2.7976199e-09, 8.4238906e-31, 4.2730667e-06, 8.4190830e-36,
       1.0934025e-29, 8.0688800e-10, 7.5980139e-11, 5.2727310e-13,
       1.0155110e-07, 9.9999559e-01, 2.3336923e-26], dtype=float32)

In [25]:
np.sum(pred[0])

0.99999994

#### Make prediction on a single image

In [47]:
from PIL import Image
import numpy as np
from skimage import transform

def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (256, 256, 1))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

image = load('test-small/1.2.826.0.1.3680043.8.498.82567018128261205674917612446318736513.jpg')
model.predict(image)

array([[0.09278035, 0.0855211 , 0.08996072, 0.08267093, 0.08524209,
        0.0948183 , 0.09885715, 0.09333548, 0.09547871, 0.09029117,
        0.09104402]], dtype=float32)